In [15]:
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random

imageDim = (960,960)
# 247500, 198000, 205500, 232000, and 136000.
# load sanitizer model
model = load_model('../FYP testing script prof gave/v3_Output/v3_1_1_4/1_1_4_205500_sanitizer.h5')
### end

# Load the cascade
face_cascade = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_default.xml')
### end

# Load Yolo skin detector model
net = cv2.dnn.readNet("../YOLOv3/11yolov3_training_last.weights", "../YOLOv3/5yolov3_testing.cfg")

# Name custom object
classes = ["C","E","H","V","A"]

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 10, size=(len(classes), 3))
### end

# To capture video from webcam.  
cap = cv2.VideoCapture(0)
# To use a video file as input 
# cap = cv2.VideoCapture('filename.mp4')

# create noise
noise_in = np.random.normal(0, 1, (1, 16, 16, 1))

# Read from image file################################.##### maxresdefaultCrop
img = cv2.imread('C:/Users/kraji/Desktop/Codes/FYP testing script prof gave/Pics/virtgirl.jpg')
print(img.shape[0])
print(img.shape[1])
oriImageDim = (int(img.shape[1]),int(img.shape[0]))
img = cv2.resize(img, imageDim, interpolation = cv2.INTER_AREA)
# END END END #####################################

while True:   
    
#     # Read the frame#####################################
#     _, img = cap.read()
# #####################################
#     # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect the faces
    faces = face_cascade.detectMultiScale(img, 1.1, 4)

    # Draw the rectangle around each face
    NumOfFaces = 0
    for (faceX, faceY, faceW, faceH) in faces:
        NumOfFaces += 1
#         cv2.rectangle(gray, (x, y), (x+w, y+h), (255, 0, 0), 2)
    if(NumOfFaces != 0):
        # crop out face
        FaceCropBuffer = 10
        FaceCrop = img[faceY:faceY+faceH+FaceCropBuffer,faceX:faceX+faceW]
        GrayFaceCrop = cv2.cvtColor(FaceCrop, cv2.COLOR_BGR2GRAY)

        ######################### SANITIZER ##############################################
        # put into sanitizer & replace noise with sanitized image. problem..: sanitizer only take in 64 by 64
        ## trying to downsize input image to 64x64. output will upsize to original size.\
        dim = (64,64)
        finalDim = (faceW,faceH+FaceCropBuffer)
        resizedFaceCrop = cv2.resize(GrayFaceCrop, dim, interpolation = cv2.INTER_AREA)

        imgResize = resizedFaceCrop.reshape(1,64,64,1) # see imgResize


        # evaluate the model
        score = model.predict([noise_in, imgResize/255])
        cropped = score[0,0:64,0:64,0]
        OUTPUT = cv2.resize(cropped, finalDim, interpolation = cv2.INTER_AREA)


        #debug
        # print(noise_in)
    #     cv2.imshow('OG', cropped)
        cv2.imshow('OUTPUT', OUTPUT)
        print(OUTPUT*255)

    
    ######################### YOLO ##############################################
    # mustr do finish by today
    #must wait to take in color image = img NEED TAKEOUT FACE
    # end goal is to make everything gray
    
    
    image = cv2.resize(img, imageDim, interpolation = cv2.INTER_AREA)
    height, width, channels = image.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(image, 0.00392, imageDim, (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN

            
    
    ######################### END ##############################################
    
    # adding block to where face is
#     noise = np.random.normal(0, 200, (faceW,faceH+FaceCropBuffer))
#     gray[y:y+h,x:x+w] = noise
#     OUPUT = np.array(np.stack((noise,noise,noise), axis=-1))
    
    if(NumOfFaces != 0):
        OUTPUT = np.array(np.stack((OUTPUT,OUTPUT,OUTPUT), axis=-1)) # to add gray imge to picture Yolo
        print(OUTPUT.shape)
        print(str(faceH+FaceCropBuffer) + " x " + str(faceW))
        print("hello")
        image[faceY:faceY+faceH+FaceCropBuffer,faceX:faceX+faceW] = OUTPUT*255
        cv2.imshow('resizedFaceCrop', resizedFaceCrop)
        cv2.resizeWindow("resizedFaceCrop", 300, 300)
    
    ### make everything greyscale so as to keep consistency in the image.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #### draw boxes YOLO #####
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 1)
            cv2.putText(image, label, (x, y + 30), font, 1, color, 1)
    
#     image = cv2.resize(image, oriImageDim, interpolation = cv2.INTER_AREA)
    print(image.shape)
    # Display 
#     cv2.imshow('FaceCrop', FaceCrop)
#     cv2.resizeWindow("FaceCrop", 300, 300)
#     cv2.namedWindow('Image', 700) #yolo
#     cv2.resizeWindow('Image', 700, 700) #yolo
    cv2.imshow('Image', image)
    
#     cv2.imshow('noise', noise)

    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
        
cv2.destroyAllWindows()
cap.release()

960
960
[[132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 [132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 [132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 ...
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]]
3
[[0]]
(454, 444, 3)
454 x 444
hello
(960, 960)
[[132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 [132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 [132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 ...
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]]
3
[[0]]
(454, 444, 3)
454 x 444
hello
(960, 960)
[[132.

3
[[0]]
(454, 444, 3)
454 x 444
hello
(960, 960)
[[132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 [132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 [132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 ...
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]]
3
[[0]]
(454, 444, 3)
454 x 444
hello
(960, 960)
[[132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 [132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 [132.72177  132.72177  132.72177  ...  89.794945  89.794945  89.794945]
 ...
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]
 [ 62.49006   62.49006   62.49006  ... 127.42463  127.42463  127.42463 ]]
3
[[0]]
(454, 

In [9]:
for i in range(5001):
    if i%500 == 0 :
        print(i)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
